In [3]:
import pandas as pd
import statsmodels.api as sm

In [4]:
batting_2024 = pd.read_csv(r"D:\GitHub\Econometrics_Term_Paper\Yearly_Data\2024batting.csv")
pitching_2024 = pd.read_csv(r"D:\GitHub\Econometrics_Term_Paper\Yearly_Data\2024pitching.csv")
fielding_2024 = pd.read_csv(r"D:\GitHub\Econometrics_Term_Paper\Yearly_Data\2024fielding.csv")
win_totals = pd.read_csv(r"D:\GitHub\Econometrics_Term_Paper\Yearly_Data\win_totals.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\GitHub\\Econometrics_Term_Paper\\Yearly_Data\\2024batting.csv'

In [ ]:
initials_dict = {"ARI": "Arizona Diamondbacks", "ATL": "Atlanta Braves", "BAL": "Baltimore Orioles", "BOS": "Boston Red Sox", "CHC": "Chicago Cubs", "CHW": "Chicago White Sox", "CIN": "Cincinnati Reds", "CLE": ["Cleveland Guardians", "Cleveland Indians"], "COL": "Colorado Rockies", "DET": "Detroit Tigers", "HOU": "Houston Astros", "KCR": "Kansas City Royals", "LAA": ["Los Angeles Angels", "Los Angeles Angels of Aneheim"], "LAD": "Los Angeles Dodgers", "MIA": "Miami Marlins", "MIL": "Milwaukee Brewers", "MIN": "Minnesota Twins", "NYM": "New York Mets", "NYY": "New York Yankees", "OAK": "Oakland Athletics", "PHI": "Philadelphia Phillies", "PIT": "Pittsburgh Pirates", "SDP": "San Diego Padres", "SFG": "San Francisco Giants", "SEA": "Seattle Mariners", "STL": "St. Louis Cardinals", "TBR": "Tampa Bay Rays", "TEX": "Texas Rangers", "TOR": "Toronto Blue Jays", "WSN": "Washington Nationals"}

In [ ]:
win_totals = win_totals.melt(
    id_vars=["Year", "G"],
    var_name="Team", 
    value_name="Wins"
    )
win_totals.dropna(inplace=True)
win_totals.reset_index(drop=True, inplace=True)
win_totals.drop("G", axis=1, inplace=True)

In [ ]:
batting_2024.set_index("Tm", inplace=True)
win_totals2024 = win_totals[(win_totals["Year"] == 2024)]